# Getting Started with Cloud-Native HLS Data in Python  

## Summary  

This tutorial was developed to examine changes in enhanced vegetation index (EVI) over an agricultural region in northern California. **The goal of the project is to observe HLS-derived mean EVI over these regions without downloading the entirety of the HLS source data.** In this notebook we will extract an EVI timeseries from Harmonized Landsat Sentinel-2 (HLS) data in the Cloud using the `earthaccess` and `rioxarray` libraries. This tutorial will show how to find the HLS data available in the cloud for our specific time period, bands (layers), and region of interest. After finding the desired data, we will load subsets of the cloud optimized geotiffs (COGs) into a Jupyter Notebook directly from the cloud, and calculate EVI. After calculating EVI we will save and stack the time series, visualize it, and export a CSV of EVI statistics for the region.  

## Background  

The  Harmonized Landsat Sentinel-2 ([HLS](https://lpdaac.usgs.gov/data/get-started-data/collection-overview/missions/harmonized-landsat-sentinel-2-hls-overview/)) project produces seamless, harmonized surface reflectance data from the Operational Land Imager (OLI) and Multi-Spectral Instrument (MSI) aboard Landsat and Sentinel-2 Earth-observing satellites, respectively. The aim is to produce seamless products with normalized parameters, which include atmospheric correction, cloud and cloud-shadow masking, geographic co-registration and common gridding, normalized bidirectional reflectance distribution function, and spectral band adjustment. This will provide global observation of the Earth’s surface every 2-3 days with 30 meter spatial resolution. One of the major applications that will benefit from HLS is agriculture assessment and monitoring, which is used as the use case for this tutorial.  

NASA's Land Processes Distributed Active Archive Center (LP DAAC) archives and distributes HLS products in the LP DAAC Cumulus cloud archive as Cloud Optimized GeoTIFFs (COG). This tutorial will demonstrate  Because these data are stored as COGs, this tutorial will teach users how to load subsets of individual files into memory for just the bands you are interested in--a paradigm shift from the more common workflow where you would need to download a .zip/HDF file containing every band over the entire scene/tile. This tutorial covers how to process HLS data (calculate EVI), visualize, and "stack" the scenes over a region of interest into an [xarray](http://xarray.pydata.org/en/stable/) data array, calculate statistics for an EVI time series, and export as a comma-separated values (CSV) file--providing you with all of the information you need for your area of interest without having to download the source data file. The Enhanced Vegetation Index ([EVI](https://earthobservatory.nasa.gov/features/MeasuringVegetation/measuring_vegetation_4.php)), is a vegetation index similar to NDVI that has been found to be more sensitive to ground cover below the vegetated canopy and saturates less over areas of dense green vegetation.  

## Requirements  

- A [NASA Earthdata Login](https://urs.earthdata.nasa.gov/) account is required to download the data used in this tutorial. You can create an account at the link provided.

- You will will also need to have a netrc file set up in your home directory in order to successfully run the code below. A code chunk in a later section provides a way to do this, or you can check out the [setup_intstructions.md](../../python/setup/setup_instructions.md).  

## Learning Objectives

- How to work with HLS Landsat ([HLSL30.002](https://doi.org/10.5067/HLS/HLSL30.002)) and Sentinel-2 ([HLSS30.002](https://doi.org/10.5067/HLS/HLSS30.002)) data products  
- How to query and subset HLS data using the `earthaccess` library  
- How to access and work with HLS data  

## Data Used  

- Daily 30 meter (m) global HLS Sentinel-2 Multi-spectral Instrument Surface Reflectance - [HLSS30.002](https://doi.org/10.5067/HLS/HLSS30.002)  
    - The HLSS30 product provides 30 m Nadir normalized Bidirectional Reflectance Distribution Function (BRDF)-Adjusted Reflectance (NBAR) and is derived from Sentinel-2A and Sentinel-2B MSI data products.  
    - Science Dataset (SDS) layers:  
        - B8A (NIR Narrow)  
        - B04 (Red)  
        - B02 (Blue)   
- Daily 30 meter (m) global HLS Landsat-8 OLI Surface Reflectance - [HLSL30.002](https://doi.org/10.5067/HLS/HLSL30.002)  
    - The HLSL30 product provides 30 m Nadir normalized Bidirectional Reflectance Distribution Function (BRDF)-Adjusted Reflectance (NBAR) and is derived from Landsat-8 OLI data products.  
     - Science Dataset (SDS) layers:  
        - B05 (NIR)  
        - B04 (Red)  
        - B02 (Blue)   

## Tutorial Outline  

1. [**Getting Started**](#getstarted)  
    1.1 Import Packages  
    1.2 EarthData Login  
2. [**Finding HLS Data**](#find)  
3. [**Accessing HLS COG Data in the Cloud**](#cloudaccess)  
    3.1 Subset by Band  
    3.2 Load COGS into Memory  
    3.3 Subset Spatially  
    3.4 Apply Scale Factor  
4. [**Processing HLS Data**](#processhls)  
    4.1 Subset by Band  
    4.2 Calculate EVI  
    4.3 Export to COG  
5. [**Automation**](#automation)  
6. [**Stacking HLS Data**](#stackhls)  
    6.1 Open and Stack COGs  
    6.2 Visualize Stacked Time Series  
7. [**Export Statistics**](#export)  
 

---

## 1. Getting Started<a id="getstarted"></a>

### 1.1 Import Packages 

Import the required packages.

In [ ]:
import os
from datetime import datetime
import requests as r
import numpy as np
import pandas as pd
import geopandas as gp
from skimage import io
import matplotlib.pyplot as plt
from osgeo import gdal
import rasterio as rio
import xarray as xr
import rioxarray as rxr
import hvplot.xarray
import hvplot.pandas
import json
import panel as pn
import geoviews
import earthaccess

### 1.2 Earthdata Login Authentication

We will use the [`earthaccess`](https://github.com/nsidc/earthaccess#readme) package for authentication. `earthaccess` can either createa a new local `.netrc` file to store credentials or validate that one exists already in you user profile. If you do not have a `.netrc` file, you will be prompted for your credentials and one will be created.  

In [ ]:
earthaccess.login(persist=True)

## 2. Finding HLS Data using `earthaccess` <a id="find"></a>

To find HLS data, we will use the `earthaccess` python library to search NASA's Common Metadata Repository (CMR) for HLS data. We will use an geojson file containing our region of interest (ROI) to search for files that intersect. To do this, we will simplify it to a bounding box. Grab the bounding coordinates from the geopandas object after opening.  

First we will read in our geojson file using `geopandas`

In [ ]:
field = gp.read_file('../../data/Field_Boundary.geojson')

We will use the `total_bounds` property to get the bounding box of our ROI, and add that to a python tuple, which is the expected data type for the bounding_box parameter `earthaccess` `search_data`.

In [ ]:
bbox = tuple(list(field.total_bounds))
bbox

When searching we can also search a specific time period of interest. Here we search from the beginning of May 2021 to the end of September 2021.

In [ ]:
temporal = ("2021-05-01T00:00:00", "2021-09-30T23:59:59")

Since the HLS collection contains to products, i.e. HLSL30 and HLSS30, we will include both short names. Search using our constraints and the `count = 100` to limit our search to 100 results.

In [ ]:
results = earthaccess.search_data(
    short_name=['HLSL30','HLSS30'],
    bounding_box=bbox,
    temporal=temporal, # 2021-07-15T00:00:00Z/2021-09-15T23:59:59Z
    count=100
)

We can preview these results in a `pandas` `dataframe` we want to check the metadata. Note we only show the first 5.

In [ ]:
pd.json_normalize(results).head(5)

We can also preview each individual results by selecting it from the list. This will show the data links, and a browse image. 

In [ ]:
results[0]

We can grab all of the URLs for the data using [`list comprehension`](https://www.w3schools.com/python/python_lists_comprehension.asp).

In [ ]:
hls_results_urls = [granule.data_links() for granule in results]
hls_results_urls

We can get the URLs for the browse images as well.

In [ ]:
browse_urls = [granule.dataviz_links()[0] for granule in results] # 0 retrieves only the https links
browse_urls

---

## 3. Accessing HLS Cloud Optimized GeoTIFFs (COGs) from Earthdata Cloud <a id="extracthls"></a>

Now that we have a list of data URLs, we will configure `gdal` and `rioxarray` to access the cloud assets that we are interested in, and read them directly into memory without needing to download the files. 

The Python libraries used to access COG files in Earthdata Cloud leverage GDAL's virtual file systems. Whether you are running this code in the Cloud or in a local workspace, GDAL configurations must be set in order to successfully access the HLS COG files.

In [ ]:
# GDAL configurations used to successfully access LP DAAC Cloud Assets via vsicurl 
gdal.SetConfigOption('GDAL_HTTP_COOKIEFILE','~/cookies.txt')
gdal.SetConfigOption('GDAL_HTTP_COOKIEJAR', '~/cookies.txt')
gdal.SetConfigOption('GDAL_DISABLE_READDIR_ON_OPEN','EMPTY_DIR')
gdal.SetConfigOption('CPL_VSIL_CURL_ALLOWED_EXTENSIONS','TIF')
gdal.SetConfigOption('GDAL_HTTP_UNSAFESSL', 'YES')

## 2.1 Subset by Band 

Lets take a look at the URLs for one of our returned granules.

In [ ]:
h = hls_results_urls[10]  
h

To calculate the EVI for each granule we need the near-infrared, red, and blue bands. Below you can find the different band numbers for each of the two products.

### Sentinel 2:
        - "narrow" NIR = B8A
        - Red = B04
        - Blue = B02  
### Landsat 8:
        - NIR = B05
        - Red = B04
        - Blue = B02  

 We will subset our URLs to include the bands identified above. 

In [ ]:
evi_band_links = []

# Define which HLS product is being accessed
if h[0].split('/')[4] == 'HLSS30.020':
    evi_bands = ['B8A', 'B04', 'B02'] # NIR RED BLUE for S30
else:
    evi_bands = ['B05', 'B04', 'B02'] # NIR RED BLUE for L30

# Subset the assets in the item down to only the desired bands
for a in h: 
    if any(b in a for b in evi_bands):
        evi_band_links.append(a)
evi_band_links

Remember from above that you can always quickly load in the browse image to get a quick view of the item using our list of browse URLs.

In [ ]:
image = io.imread(browse_urls[10])  # Load jpg browse image into memory

# Basic plot of the image
plt.figure(figsize=(10,10))              
plt.imshow(image)
plt.show()

Above, we see our first observation over the northern Central Valley of California. 

In [ ]:
del image # Remove the browse image

## 2.2 Load HLS COGs into Memory 

HLS COGs are broken into chunks allowing data to be read more efficiently. Define the chunk size of an HLS tile, mask the NaN values, then read the files using `rioxarray` and name them based upon the band. We also squeeze the object to remove the band dimension from most of the files, since there is only 1 band.

> **NOTE:** To scale the bands, you can set the `mask_and_scale` to `True` (`mask_and_scale=True`), however the `scale_factor` information in some of the HLSL30 granules are found in the `file` metadata, but missing from the `Band` metadata. `rioxarray` looks for the `scale_factor` under `Band` metadata and if this information is missing it assumes the `scale_factor` is equal to 1. This results in having data to be uscaled and not masked for those granules. That is why we treat our data a bit differently here, leaving it unscaled and manually updating the `scale_factor` attribute in the `xarray` `dataarray`.   

In [ ]:
# Use vsicurl to load the data directly into memory (be patient, may take a few seconds)
chunk_size = dict(band=1, x=512, y=512) # Tiles have 1 band and are divided into 512x512 pixel chunks
# Sometimes a vsi curl error occurs so we need to retry if it does
max_retries = 10
for e in evi_band_links:
    print(e)
    # Try Loop
    for _i in range(max_retries):
        try:
            # Open and build datasets
            if e.rsplit('.', 2)[-2] == evi_bands[0]:      # NIR index
                nir = rxr.open_rasterio(e, chunks=chunk_size, masked=True).squeeze('band', drop=True)
                nir.attrs['scale_factor'] = 0.0001        # hard coded the scale_factor attribute 
            elif e.rsplit('.', 2)[-2] == evi_bands[1]:    # red index
                red = rxr.open_rasterio(e, chunks=chunk_size, masked=True).squeeze('band', drop=True)
                red.attrs['scale_factor'] = 0.0001        # hard coded the scale_factor attribute
            elif e.rsplit('.', 2)[-2] == evi_bands[2]:    # blue index
                blue = rxr.open_rasterio(e, chunks=chunk_size, masked=True).squeeze('band', drop=True)
                blue.attrs['scale_factor'] = 0.0001       # hard coded the scale_factor attribute
            break # Break out of the retry loop
        except Exception as ex:
            print(f"vsi curl error: {ex}. Retrying...")
    else:
        print(f"Failed to process {e} after {max_retries} retries. Please check to see you're authenticated with earthaccess.")
print("The COGs have been loaded into memory!")

> **NOTE:** Getting an error in the section above? Accessing these files in the cloud requires you to authenticate using your NASA Earthdata Login account. You will need to have a netrc file set up containing those credentials in your home directory in order to successfully run the code below. Please make sure you have a valid username and password in the created netrc file. 

We can take a quick look at one of the `dataarray` we just read in.

In [ ]:
nir

**Note** the full size of the array, **y**=3660 & **x**=3660

## 2.3 Subset spatially

Before we can subset using our input farm field, we will first need to convert the `geopandas` dataframe from lat/lon (EPSG: 4326) into the [projection used by HLS](https://lpdaac.usgs.gov/data/get-started-data/collection-overview/missions/harmonized-landsat-sentinel-2-hls-overview/#hls-tiling-system), UTM (aligned to the Military Grid Reference System). Since UTM is a zonal projection, we'll need to extract the unique UTM zonal projection parameters from our input HLS files and use them to transform the coordinate of our input farm field. 

We can print out the WKT string for our HLS tiles.

In [ ]:
nir.spatial_ref.crs_wkt

We will use this information to transform the coordinates of our ROI to the proper UTM projection.

In [ ]:
fsUTM = field.to_crs(nir.spatial_ref.crs_wkt) # Take the CRS from the NIR tile that we opened and apply it to our field geodataframe.
fsUTM

Now, we can use our field ROI to mask any pixels that fall outside of it and crop to the bounding box using `rasterio`. This greatly reduces the amount of data that are needed to load into memory. 

In [ ]:
nir_cropped = nir.rio.clip(fsUTM.geometry.values, fsUTM.crs, all_touched=True) # All touched includes any pixels touched by the polygon
nir_cropped

**Note** that the array size is considerably smaller than the full size we read in before.

Now we will plot the cropped NIR data.

In [ ]:
nir_cropped.hvplot.image(aspect='equal', cmap='viridis', frame_width= 800, fontscale=1.6, geo=True, tiles='ESRI').opts(title='HLS Cropped NIR Band', )  # Quick visual to assure that it worked

Above, you can see that the data have been loaded into memory already subset to our ROI. Also notice that the data has not been scaled (see the legend). We will next scaled the data using the function defined below. 

## 2.4 Apply Scale Factor 

In [ ]:
# Define function to scale 
def scaling(band):
    scale_factor = band.attrs['scale_factor'] 
    band_out = band.copy()
    band_out.data = band.data*scale_factor
    band_out.attrs['scale_factor'] = 1
    return(band_out)

In [ ]:
nir_cropped_scaled = scaling(nir_cropped)

We can plot to confirm our manual scaling worked.

In [ ]:
nir_cropped_scaled.hvplot.image(aspect='equal', cmap='viridis', frame_width= 800, fontscale=1.6, geo=True, tiles='ESRI').opts(title='HLS Cropped NIR Band')  # Quick visual to assure that it worked

Next, load in the red and blue bands and fix their scaling as well.

In [ ]:
# Red
red_cropped = red.rio.clip(fsUTM.geometry.values, fsUTM.crs, all_touched=True)
red_cropped_scaled = scaling(red_cropped)
# Blue
blue_cropped = blue.rio.clip(fsUTM.geometry.values, fsUTM.crs, all_touched=True)
blue_cropped_scaled = scaling(blue_cropped)
print('Data is loaded and scaled!')

---
## 4. Processing HLS Data <a id="processhls"></a>

In this section we will use the HLS data we've access to calculate the EVI. We will do this by defining a function to calculate EVI that will retain the attributes and metadata associated with the data we accessed.

### 4.1 Calculate EVI

Below is a function we'll use to calculate EVI using the NIR, Red, and Blue bands. The function will: 
1. build a new `xarray` `dataarray` with EVI values
2. copy the original file metadata to the new `xarray` `dataarray`

In [ ]:
def calc_evi(red, blue, nir):
      # Create EVI xarray.DataArray that has the same coordinates and metadata
      evi = red.copy()
      # Calculate the EVI
      evi_data = 2.5 * ((nir.data - red.data) / (nir.data + 6.0 * red.data - 7.5 * blue.data + 1.0))
      # Replace the Red xarray.DataArray data with the new EVI data
      evi.data = evi_data
      # exclude the inf values
      evi = xr.where(evi != np.inf, evi, np.nan, keep_attrs=True)
      # change the long_name in the attributes
      evi.attrs['long_name'] = 'EVI'
      evi.attrs['scale_factor'] = 1
      return evi

Below, apply the EVI function on the scaled data.

In [ ]:
evi_cropped = calc_evi(red_cropped_scaled, blue_cropped_scaled, nir_cropped_scaled) # Generate EVI array
evi_cropped


Next, plot the results using `hvplot`.

In [ ]:
evi_cropped.hvplot.image(aspect='equal', cmap='YlGn', frame_width= 800, fontscale=1.6, geo=True, tiles='ESRI').opts(title=f'HLS-derived EVI, {evi_cropped.SENSING_TIME}')

Above, notice that variation of green level appearing in different fields in our ROI, some being much greener than the others. 

### 5 Export to COG 

In this section, create an output filename and export the cropped EVI to COG. 

In [ ]:
original_name = evi_band_links[0].split('/')[-1]
original_name

The standard format  for HLS S30 V2.0 and HLS L30 V2.0 filenames is as follows:
> **HLS.S30/HLS.L30**: Product Short Name    
**T10TEK**: MGRS Tile ID (T+5-digits)  
**2020273T190109**: Julian Date and Time of Acquisition (YYYYDDDTHHMMSS)  
**v2.0**: Product Version   
**B8A/B05**: Spectral Band  
**.tif**: Data Format (Cloud Optimized GeoTIFF)  

For additional information on HLS naming conventions, be sure to check out the [HLS Overview Page](https://lpdaac.usgs.gov/data/get-started-data/collection-overview/missions/harmonized-landsat-sentinel-2-hls-overview/#hls-naming-conventions).

Now modify the filename to describe that its EVI, cropped to an ROI.

In [ ]:
out_name = f"{original_name.split('v2.0')[0]}v2.0_EVI_cropped.tif"  # Generate output name from the original filename
out_name

Use the `COG` driver to write a local raster output. A cloud-optimized geotiff (COG) is a geotiff file that has been tiled and includes overviews so it can be accessed and previewed without having to load the entire image into memory at once.

In [ ]:
out_folder = '../../data/'
evi_cropped.rio.to_raster(raster_path=f'{out_folder}{out_name}', driver='COG')

In [ ]:
del evi_cropped, out_folder, out_name, red_cropped, blue_cropped, nir_cropped, red_cropped_scaled, blue_cropped_scaled, nir_cropped_scaled

---

## 5. Automation <a id="automation"></a>

In this section, automate sections 4-5 for each HLS item that intersects our spatiotemporal subset of interest. Loop through each item and subset to the desired bands, load the spatial subset into memory, apply the scale factor, calculate EVI, and export as a Cloud Optimized GeoTIFF. 

In [ ]:
len(hls_results_urls)

Now put it all together and loop through each of the files, visualize and export cropped EVI files. 
Be patient with the for loop below, it may take a few minutes to complete. 

In [ ]:
for j, h in enumerate(hls_results_urls):
    
    outName = h[0].split('/')[-1].split('v2.0')[0] +'v2.0_EVI_cropped.tif'
    print(outName)

    evi_band_links = []
    if h[0].split('/')[4] == 'HLSS30.020':
        evi_bands = ['B8A', 'B04', 'B02'] # NIR RED BLUE
    else:
        evi_bands = ['B05', 'B04', 'B02'] # NIR RED BLUE
    
    for a in h: 
        if any(b in a for b in evi_bands):
            evi_band_links.append(a)

    
    # Check if file already exists in output directory, if yes--skip that file and move to the next observation
    if os.path.exists(f'../../data/{outName}'):
        print(f"{outName} has already been processed and is available in this directory, moving to next file.")
        continue
    
    # Use vsicurl to load the data directly into memory (be patient, may take a few seconds)
    chunk_size = dict(band=1, x=512, y=512) # Tiles have 1 band and are divided into 512x512 pixel chunks
    # Sometimes a vsi curl error occurs so we need to retry if it does
    max_retries = 10
    for e in evi_band_links:
        print(e)
        # Try Loop
        for _i in range(max_retries):
            try:
                if e.rsplit('.', 2)[-2] == evi_bands[0]:      # NIR index
                    nir = rxr.open_rasterio(e, chunks=chunk_size, masked= True).squeeze('band', drop=True)
                    nir.attrs['scale_factor'] = 0.0001                         # hard coded the scale_factor attribute 
                elif e.rsplit('.', 2)[-2] == evi_bands[1]:    # red index
                    red = rxr.open_rasterio(e, chunks=chunk_size, masked= True).squeeze('band', drop=True)
                    red.attrs['scale_factor'] = 0.0001                         # hard coded the scale_factor attribute
                elif e.rsplit('.', 2)[-2] == evi_bands[2]:    # blue index
                    blue = rxr.open_rasterio(e, chunks=chunk_size, masked= True).squeeze('band', drop=True)
                    blue.attrs['scale_factor'] = 0.0001                        # hard coded the scale_factor attribute
                break # Break out of the retry loop
            except Exception as ex:
                print(f"vsi curl error: {ex}. Retrying...")
        else:
            print(f"Failed to process {e} after {max_retries} retries. Please check to see you're authenticated with earthaccess.")
        
    fsUTM = field.to_crs(nir.spatial_ref.crs_wkt)

    # Crop to our ROI and apply scaling and masking
    nir_cropped = nir.rio.clip(fsUTM.geometry.values, fsUTM.crs, all_touched=True)
    red_cropped = red.rio.clip(fsUTM.geometry.values, fsUTM.crs, all_touched=True)
    blue_cropped = blue.rio.clip(fsUTM.geometry.values, fsUTM.crs, all_touched=True)
    
    print('Cropped')      
    
    # Fix Scaling
    nir_cropped_scaled = scaling(nir_cropped)
    red_cropped_scaled = scaling(red_cropped)
    blue_cropped_scaled = scaling(blue_cropped)

    # Generate EVI
    
    evi_cropped = calc_evi(red_cropped_scaled, blue_cropped_scaled, nir_cropped_scaled)

    print('EVI Calculated')
    
    # Remove any observations that are entirely fill value
    if np.nansum(evi_cropped.data) == 0.0:
        print(f"File: {h[0].split('/')[-1].rsplit('.', 1)[0]} was entirely fill values and will not be exported.")
        continue
        
    evi_cropped.rio.to_raster(raster_path=f'../../data/{outName}', driver='COG')
    
    print(f"Processed file {j+1} of {len(hls_results_urls)}")
    

Now there should be multiple COGs exported to your working directory, that will be used in Section 6 to stack into a time series. 

---

## 6. Stacking HLS Data <a id="stackhls"></a>

In this section we will open multiple HLS-derived EVI COGs and stack them into an `xarray` data array along the time dimension. First list the files we created in the `/data/` directory.

### 6.1 Open and Stack COGs

In [ ]:
evi_dir = '../../data/'
evi_files = [os.path.abspath(os.path.join(evi_dir, o)) for o in os.listdir(evi_dir) if o.endswith('EVI_cropped.tif')]  # List EVI COGs
evi_files

Create a time index as an xarray variable from the filenames.

In [ ]:
def time_index_from_filenames(evi_files):
    '''
    Helper function to create a pandas DatetimeIndex
    '''
    return [datetime.strptime(f.split('.')[-4], '%Y%jT%H%M%S') for f in evi_files]

time = xr.Variable('time', time_index_from_filenames(evi_files))

Next, the cropped HLS COG files are being read using `rioxarray` and a time series stack is created using `xarray`. 

In [ ]:
chunks=dict(band=1, x=512, y=512)

evi_ts = xr.concat([rxr.open_rasterio(f, mask_and_scale=True, chunks=chunks).squeeze('band', drop=True) for f in evi_files], dim=time)
evi_ts.name = 'EVI'

In [ ]:
evi_ts = evi_ts.sortby(evi_ts.time)
evi_ts

### 6.2 Visualize Stacked Time Series

Below, use the [`hvPlot`](https://hvplot.pyviz.org/index.html) and [`holoviews`](https://www.holoviews.org/) packages to create an interactive time series plot of the HLS derived EVI data.Basemap layer is also added to provide better context of the areas surrounding our region of interest.

In [ ]:
# This cell generates a warning
import warnings
warnings.filterwarnings('ignore')


# set the x, y, and z (groupby) dimensions, add a colormap/bar and other parameters.
title = 'HLS-derived EVI over agricultural fields in northern California'
evi_ts.hvplot.image(x='x', y='y', groupby = 'time', frame_height=500, frame_width= 500, cmap='YlGn', geo=True, tiles = 'EsriImagery')

Looking at the time series above, the farm fields are pretty stable in terms of EVI during our temporal range. The slow decrease in EVI as we move toward Fall season could show these fields are having some sort of trees rather than crops. I encourage you to expand your temporal range to learn more about the EVI annual and seasonal changes. 

Since the data is in an xarray we can intuitively slice or reduce the dataset. Let's select a single time slice from the EVI variable.

You can use slicing to plot data only for a specific observation, for example.

In [ ]:
title = 'HLS-derived EVI over agricultural fields in northern California'
# evi_cropped.hvplot.image(aspect='equal', cmap='YlGn', frame_width=300).opts(title=f'HLS-derived EVI, {evi_cropped.SENSING_TIME}', clabel='EVI')

evi_ts.isel(time=1).hvplot.image(x='x', y='y', cmap='YlGn', geo=True, tiles = 'EsriImagery', frame_height=500, frame_width= 500).opts(title=f'{title}, {evi_ts.isel(time=4).SENSING_TIME}')


Now, plot the time series as boxplots showing the distribution of EVI values for our farm field.

In [ ]:
evi_ts.hvplot.box('EVI', by=['time'], rot=90, box_fill_color='lightblue', width=800, height=600).opts(ylim=(-0.5,1.5))

The statistics shows a relatively stable green status in these fields during mid May to the end of September 2021. 

## 7. Export Statistics<a id="export"></a>

Next, calculate statistics for each observation and export to CSV. 

In [ ]:
# xarray allows you to easily calculate a number of statistics
evi_min = evi_ts.min(('y', 'x'))
evi_max = evi_ts.max(('y', 'x'))
evi_mean = evi_ts.mean(('y', 'x'))
evi_sd = evi_ts.std(('y', 'x'))
evi_count = evi_ts.count(('y', 'x'))
evi_median = evi_ts.median(('y', 'x'))

We now have the `mean` and `standard deviation` for each time slice as well as the `maximum` and `minimum` values. Let's do some plotting! We will use the [`hvPlot`](https://hvplot.pyviz.org/index.html) package to create simple but interactive charts/plots. Hover your curser over the visualization to see the data values. 

In [ ]:
evi_mean.hvplot.line()

In [ ]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Combine line plots for different statistics
stats = (evi_mean.hvplot.line(height=350, width=450, line_width=1.5, color='red', grid=True, padding=0.05).opts(title='Mean')+ 
    evi_sd.hvplot.line(height=350, width=450, line_width=1.5, color='red', grid=True, padding=0.05).opts(title='Standard Deviation')
    + evi_max.hvplot.line(height=350, width=450, line_width=1.5, color='red', grid=True, padding=0.05).opts(title='Max') + 
    evi_min.hvplot.line(height=350, width=450, line_width=1.5, color='red', grid=True, padding=0.05).opts(title='Min')).cols(2)
stats

Remember that these graphs are also interactive--hover over the line to see the value for a given date. 

Finally, create a `pandas` dataframe with the statistics, and export to a CSV file. 

In [ ]:
# Create pandas dataframe from dictionary
df = pd.DataFrame({'Min EVI': evi_min, 'Max EVI': evi_max, 
                   'Mean EVI': evi_mean, 'Standard Deviation EVI': evi_sd, 
                   'Median EVI': evi_median, 'Count': evi_count})

In [ ]:
df.index = evi_ts.time.data                       # Set the observation date as the index
df.to_csv('../../data/HLS-Derived_EVI_Stats.csv', index=True)  # Export to CSV

Success! You have now not only learned how to get started with HLS V2.0 data, but have also learned how to navigate cloud-native data `earthaccess`, how to access subsets of COGs, and how to write COGs for your own outputs. Using this jupyter notebook as a workflow, you should now be able to switch to your specific region of interest and re-run the notebook. Good Luck!

## Contact Info  

Email: LPDAAC@usgs.gov  
Voice: +1-866-573-3222  
Organization: Land Processes Distributed Active Archive Center (LP DAAC)¹  
Website: <https://lpdaac.usgs.gov/>  
Date last modified: 01-17-2024  

¹Work performed under USGS contract G15PD00467 for NASA contract NNG14HH33I. 